In [1]:
import os

import numpy as np
import pandas as pd
# Torch
import torch
import torchvision
from torchvision import datasets,transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib
from torchvision import models

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
transformer = transforms.Compose([
    # Resize the image to (160, 160)
    transforms.Resize((160, 160)),
    # Convert the image to grayscale only one channel
    transforms.Grayscale(num_output_channels=1),

    #Data Augmentation
    transforms.RandomHorizontalFlip(),       # Randomly flip the image horizontally
    transforms.RandomRotation(degrees=15),   # Randomly rotate the image by up to 15 degrees
    transforms.ToTensor(),                   # Convert the image to a PyTorch tensor
])


In [5]:

test_path = '/content/drive/MyDrive/Colab Notebooks/DL Submission/test'
test_loader = DataLoader(torchvision.datasets.ImageFolder(
                          test_path,
                          transform=transformer),
                          batch_size = 128,
                          shuffle=False)

In [6]:
class ConvNet(nn.Module):
  def __init__(self,num_classes=2):
    super(ConvNet,self).__init__()
    self.conv1 =nn.Conv2d(in_channels=1,out_channels=5,kernel_size=5,stride=1,padding=2) # outptu channels is the number of kernels used
    self.bn1 = nn.BatchNorm2d(num_features=5)
    self.sigmoid1 = nn.Sigmoid()

    #256*20*80*80
    self.conv2 = nn.Conv2d(in_channels=5,out_channels = 16,kernel_size =5,stride=1,padding=2)
    self.bn2=nn.BatchNorm2d(num_features=16)
    self.pool1=nn.MaxPool2d(kernel_size=2)
    self.sigmoid2 = nn.Sigmoid()

    self.conv3 = nn.Conv2d(in_channels=16,out_channels =32,kernel_size =3,stride =1 ,padding=1)
    self.bn3 = nn.BatchNorm2d(num_features=32)
    self.dropout1 = nn.Dropout(0.2)
    self.sigmoid3 =nn.Sigmoid()

    self.conv4 = nn.Conv2d(in_channels=32,out_channels =50,kernel_size =3,stride =1 ,padding=1)
    self.bn4 = nn.BatchNorm2d(num_features=50)
    self.pool2=nn.MaxPool2d(kernel_size=2)
    self.sigmoid4 =nn.Sigmoid()

    self.conv5 = nn.Conv2d(in_channels=50,out_channels =70,kernel_size =3,stride =1 ,padding=1)
    self.bn5 = nn.BatchNorm2d(num_features=70)
    self.sigmoid5 =nn.Sigmoid()


    self.fc = nn.Linear(in_features=70*40*40,out_features = 2)

  def forward(self,input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.sigmoid1(output)

    output = self.conv2(output)
    output = self.bn2(output)
    output = self.pool1(output)
    output = self.sigmoid2(output)

    output = self.conv3(output)
    output = self.bn3(output)
    output = self.sigmoid3(output)

    output = self.conv4(output)
    output = self.bn4(output)
    output = self.pool2(output)
    output = self.sigmoid4(output)

    output = self.conv5(output)
    output = self.bn5(output)
    output = self.sigmoid5(output)

    output = output.view(-1,70*40*40)
    output = self.fc(output)
    return output





In [7]:
model = ConvNet(num_classes=2).to(device)

Testing on test data(unseen data)

In [11]:
#loading saved model

model_path = '/content/drive/MyDrive/Colab Notebooks/DL Submission/Chosen_Best_model_CNN2_Sig_adagrad_0.2_128_0.001.pt'
state_dict = torch.load(model_path,map_location=device)
# Load the state dictionary into the model
model.load_state_dict(state_dict)
model.to(device)
predicted_list = []
target_list =[]
# Set the model to evaluation mode
model.eval()

total_correct = 0
total_images_test = len(test_loader.dataset)
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()
        predicted_list.extend(predicted.cpu().numpy())
        target_list.extend(targets.cpu().numpy())

accuracy = total_correct / total_images_test
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 87.98%


In [12]:
from sklearn.metrics import precision_score, f1_score

# Calculate precision
precision = precision_score(target_list, predicted_list, average='macro')
print(f'Precision: {precision * 100:.2f}%')

# Calculate F1 score
f1 = f1_score(target_list, predicted_list, average='macro')
print(f'F1 Score: {f1 * 100:.2f}%')

Precision: 87.67%
F1 Score: 86.98%


Testing Single Images

In [ ]:
import torch
from PIL import Image


# Load the image
image_path = '/content/drive/MyDrive/Colab Notebooks/good_lungs.png'
image = Image.open(image_path)

# Apply transformations
input_image = transformer(image).unsqueeze(0)  # Add a batch dimension

# Load the model
model = ConvNet(num_classes=2)
model_path = '/content/drive/MyDrive/Colab Notebooks/DL Submission/CNN2_Adagrad_0.001_128'
state_dict = torch.load(model_path,map_location=device)
model.load_state_dict(state_dict)
model.eval()

# Make prediction
with torch.no_grad():
    output = model(input_image)
    _, predicted_class = torch.max(output, 1)

# Print prediction
if predicted_class==0:
  print('NORMAL')
if predicted_class==1:
  print('PNEMONIA')

NORMAL
